In [ ]:
import pandas as pd

df1 = pd.read_csv("fake_job_postings.csv")
df1.drop(['benefits', 'company_profile', 'employment_type', 'salary_range',
          'industry', 'department', 'required_experience', 'required_education', 'job_id', 'function',], axis=1, inplace=True)
df2 = pd.read_csv("job_train.csv")
df2 = df2[df2['fraudulent']==1]
merged_df = pd.concat([df1, df2])
# merged_df.to_csv("final_data.csv")
merged_df.head()

,title,location,description,requirements,telecommuting,has_company_logo,has_questions,fraudulent
0,Marketing Intern,"US, NY, New York","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever","Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,0
3,Account Executive - Washington DC,"US, DC, Washington",THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,0
4,Bill Review Manager,"US, FL, Fort Worth",JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,0


In [54]:
import unicodedata

def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalize to NFD (decompose accented chars), then filter out combining marks
    return ''.join(c for c in unicodedata.normalize('NFD', str(text))
                   if unicodedata.category(c) != 'Mn')


def clean_accents(column, print_change):
    cleaned_lst = []
    for loc in merged_df[column]:
        if not pd.isna(loc) and str(loc).strip() != "": 
            cleaned_loc = remove_accents(loc)
            if loc != cleaned_loc:
                cleaned_lst.append((loc, cleaned_loc))
    cleaned_lst = list(set(cleaned_lst))
    if print_change:
        for cleaned in cleaned_lst:
            print(cleaned)
    

text_cols = ['title', 'location', 'description', 'requirements']
for col in text_cols:
    clean_accents(col, False)

In [6]:
# Assuming your dataframe is called 'df' and column is 'text_column'
def find_unicode_chars(text_series):
    """Find all unique non-ASCII unicode characters in a text series"""
    unicode_chars = set()
    
    for text in text_series.dropna():
        for char in str(text):
            # Check if character is non-ASCII (ord > 127)
            if ord(char) > 127:
                unicode_chars.add(char)
    
    return sorted(unicode_chars, key=lambda x: ord(x))

# Get unique unicode characters
unicode_list = find_unicode_chars(merged_df['description_and_requirements'])
unicode_list = find_unicode_chars(merged_df['location'])

# Print them with their unicode codes and names
import unicodedata

print("Unique Unicode Characters Found:")
print("-" * 60)
for char in unicode_list:
    try:
        name = unicodedata.name(char)
    except ValueError:
        name = "Unknown"
    print(f"'{char}' | U+{ord(char):04X} | {name}")

Unique Unicode Characters Found:
------------------------------------------------------------
' ' | U+00A0 | NO-BREAK SPACE
'Ø' | U+00D8 | LATIN CAPITAL LETTER O WITH STROKE
'á' | U+00E1 | LATIN SMALL LETTER A WITH ACUTE
'ã' | U+00E3 | LATIN SMALL LETTER A WITH TILDE
'ä' | U+00E4 | LATIN SMALL LETTER A WITH DIAERESIS
'é' | U+00E9 | LATIN SMALL LETTER E WITH ACUTE
'ó' | U+00F3 | LATIN SMALL LETTER O WITH ACUTE
'ö' | U+00F6 | LATIN SMALL LETTER O WITH DIAERESIS
'ø' | U+00F8 | LATIN SMALL LETTER O WITH STROKE
'ü' | U+00FC | LATIN SMALL LETTER U WITH DIAERESIS
'İ' | U+0130 | LATIN CAPITAL LETTER I WITH DOT ABOVE
'ł' | U+0142 | LATIN SMALL LETTER L WITH STROKE
'ń' | U+0144 | LATIN SMALL LETTER N WITH ACUTE
'Α' | U+0391 | GREEK CAPITAL LETTER ALPHA
'Ε' | U+0395 | GREEK CAPITAL LETTER EPSILON
'Η' | U+0397 | GREEK CAPITAL LETTER ETA
'Ι' | U+0399 | GREEK CAPITAL LETTER IOTA
'Κ' | U+039A | GREEK CAPITAL LETTER KAPPA
'Λ' | U+039B | GREEK CAPITAL LETTER LAMDA
'Ν' | U+039D | GREEK CAPITAL LETTER NU

In [ ]:
# Option 1: Replace specific characters with their text equivalents
replacements = {
    '—': '--',        # em dash
    '–': '-',         # en dash
    '"': '"',         # smart quotes
    '"': '"',
    ''': "'",
    ''': "'",
    '…': '...',       # ellipsis
    '®': '(R)',
    '©': '(C)',
    '™': '(TM)',
}

df['text_column'] = df['text_column'].str.replace('|'.join(replacements.keys()), 
                                                    lambda m: replacements[m.group()], 
                                                    regex=True)

# Option 2: Remove accents from letters (keeping base letters)
import unicodedata

def remove_accents(text):
    if pd.isna(text):
        return text
    # Normalize to NFD (decompose accented chars), then filter out combining marks
    return ''.join(c for c in unicodedata.normalize('NFD', str(text))
                   if unicodedata.category(c) != 'Mn')

df['text_column'] = df['text_column'].apply(remove_accents)

# Option 3: Convert to closest ASCII equivalent
df['text_column'] = df['text_column'].str.encode('ascii', errors='ignore').str.decode('ascii')

In [3]:
# print(merged_df.isnull().sum())
print(df1.duplicated().sum())
print(df2.duplicated().sum())
print(merged_df.duplicated().sum())

df1_unique = df1.drop_duplicates()
df2_unique = df2.drop_duplicates()
overlap_count = pd.merge(df1_unique, df2_unique, how='inner').shape[0]
print(overlap_count)
print(len(df2))

397
10
852
446
456


In [4]:
merged_df.info()
print(merged_df.nunique())
merged_df.duplicated().sum()
merged_df.drop_duplicates(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 18334 entries, 0 to 8938
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   title                         18334 non-null  object
 1   location                      17980 non-null  object
 2   description                   18334 non-null  object
 3   requirements                  15564 non-null  object
 4   telecommuting                 18334 non-null  int64 
 5   has_company_logo              18334 non-null  int64 
 6   has_questions                 18334 non-null  int64 
 7   fraudulent                    18334 non-null  int64 
 8   has_requirements              18334 non-null  bool  
 9   description_and_requirements  18334 non-null  object
 10  country_state                 15735 non-null  object
 11  country                       17980 non-null  object
 12  has_location                  18334 non-null  bool  
 13  has_location_details  

In [5]:
textual = ['title', 'company_profile', 'description',
       'requirements', 'benefits']
categorical = ['location', 'employment_type', 'industry']
continuous = ['salary_range']
label = ['fraudulent']

merged_df.isnull().sum()

title                              0
location                         341
description                        0
requirements                    2637
telecommuting                      0
has_company_logo                   0
has_questions                      0
fraudulent                         0
has_requirements                   0
description_and_requirements       0
country_state                   2509
country                          341
has_location                       0
has_location_details               0
dtype: int64

In [ ]:
# print(merged_df.loc[merged_df['location_has_non_ascii'] == 1, 'location'].drop_duplicates())

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Combine relevant text columns (you can modify this based on which columns you want to use)
merged_df['text'] = merged_df['title'].fillna('') + ' ' + merged_df['description'].fillna('')

# Split the data
X = merged_df['text']
y = merged_df['fraudulent']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vec, y_train)

# Make predictions and print classification report
y_pred = model.predict(X_test_vec)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(model.coef_)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3336
           1       0.98      0.30      0.46       161

    accuracy                           0.97      3497
   macro avg       0.97      0.65      0.72      3497
weighted avg       0.97      0.97      0.96      3497

[[ 0.67765924  1.34207408 -0.17258051 ... -0.18184741 -0.0522385
  -0.0719484 ]]


In [6]:

# len(df3)
# len(merged_df)

In [7]:
merged_df.drop_duplicates(inplace=True)
# Combine relevant text columns (you can modify this based on which columns you want to use)
merged_df['text'] = merged_df['title'].fillna('') + ' ' + merged_df['description'].fillna('')

# Split the data
X = merged_df['text']
y = merged_df['fraudulent']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train logistic regression model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_vec, y_train)

# Make predictions and print classification report
y_pred = model.predict(X_test_vec)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(model.coef_)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3321
           1       0.95      0.35      0.51       176

    accuracy                           0.97      3497
   macro avg       0.96      0.67      0.75      3497
weighted avg       0.97      0.97      0.96      3497

[[ 0.28998688  1.26500827 -0.21935576 ... -0.17891268 -0.05137752
  -0.06817463]]


In [8]:
merged_df.duplicated().sum()

np.int64(0)

In [1]:
import pandas as pd

df1 = pd.read_csv("fake_job_postings.csv")
df1.drop(['benefits', 'company_profile', 'employment_type', 'salary_range', 'industry', 'department', 'required_experience', 'required_education', 'job_id', 'function',], axis=1, inplace=True)
df2 = pd.read_csv("job_train.csv")
merged_df = pd.concat([df1, df2])

merged_df.head()

,title,location,description,requirements,telecommuting,has_company_logo,has_questions,fraudulent
0,Marketing Intern,"US, NY, New York","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0,1,0,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0,1,0,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever","Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0,1,0,0
3,Account Executive - Washington DC,"US, DC, Washington",THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0,1,0,0
4,Bill Review Manager,"US, FL, Fort Worth",JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0,1,1,0


In [12]:
# print(merged_df.isnull().sum())
print(df1.duplicated().sum())
print(df2.duplicated().sum())
print(merged_df.duplicated().sum())

df1_unique = df1.drop_duplicates()
df2_unique = df2.drop_duplicates()
overlap_count = pd.merge(df1_unique, df2_unique, how='inner').shape[0]
print(overlap_count)
print(len(df2))

314
105
9337
8921
8940


In [ ]:
##### Oct 30 Flora #####

# Explore "description"
missing_desc = merged_df[merged_df['description'].isna()]
# print(missing_desc[['description', 'fraudulent', 'requirements', 'location']])

### Explore "location" ###
# Extract country and state
pattern1 = r'(^[A-Z]{2},\s*[A-Z0-9]{1,3})'
merged_df['country_state'] = merged_df['location'].str.extract(pattern1, expand=False)
# Extract country
pattern2 = r'(^[A-Z]{2})'
merged_df['country'] = merged_df['location'].str.extract(pattern2, expand=False)
# Manage Remote jobs
merged_df['is_remote'] = merged_df['location'].str.lower().str.contains('remote|work from home', na=False) & merged_df['country_state'].isna()
merged_df.loc[merged_df['is_remote'], 'country_state'] = "Remote"
merged_df.loc[merged_df['is_remote'], 'country'] = "Remote"
merged_df.drop(columns=["is_remote"], inplace=True)

# New column - location mask
merged_df["has_location"] = merged_df["location"].notna()
# New column - detailed location mask - something beyond just the country code
merged_df["has_location_details"] = (merged_df["location"].str.lower().str.strip() == merged_df["country"].str.lower().str.strip()).fillna(False)


# unique_outliers = outliers[["location"]].drop_duplicates()
# print("Outlier rows:")
# print(len(outliers["location"].unique()))
# # unique_outliers.to_csv("location_outliers")
# print(unique_outliers)

42       US
173      US
230      US
368      US
392      US
         ..
17816    US
2566     US
4370     US
5584     GB
6858     BH
Name: location, Length: 98, dtype: object


In [ ]:
# Make new feature that has binary value for whether requirements was missing or not
merged_df["has_requirements"] = merged_df["requirements"].notna()
print(merged_df["has_requirements"].head(10))

# Merge description with requirements so they are in one new feature - called description_and_requirements
merged_df["description_and_requirements"] = merged_df["description"].fillna("") + merged_df["requirements"].fillna("")
print(merged_df["description_and_requirements"].head(5))

0     True
1     True
2     True
3     True
4     True
5    False
6     True
7     True
8     True
9     True
Name: has_requirements, dtype: bool
0    Food52, a fast-growing, James Beard Award-winn...
1    Organised - Focused - Vibrant - Awesome!Do you...
2    Our client, located in Houston, is actively se...
3    THE COMPANY: ESRI – Environmental Systems Rese...
4    JOB TITLE: Itemization Review ManagerLOCATION:...
Name: description_and_requirements, dtype: object
